In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

In [ ]:
cid = open('cid.txt', 'r').readline()
secret = open('secret.txt', 'r').readline()

In [ ]:
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [ ]:
username_dict = {"jimmyjhickey": "Jimmy",
                 "12101252392": "Lukas",
                 "aeolus9": "Ross",
                 "wdiedrick243": "Will",
                 "redtranscendent": "Michael",
                 "pboelter0705": "Paul",
                 "gabemcb": "Gabe"}

In [ ]:
shared_urls = ["https://open.spotify.com/playlist/23P7MTXWrqZkaqUAzegqvI", # gathers no moss
               "https://open.spotify.com/playlist/1u5KPMOFnOvWRlxyW13VFD", # chill 90s hiphop
               "https://open.spotify.com/playlist/1pfolyLLz4pQ3jLRYQk7d5", # under the covers
               "https://open.spotify.com/playlist/1SQjekEC7UpaFPHzidKQTd", # songs with stawry
               "https://open.spotify.com/playlist/3rWCGvc62Ea50aPvdj9iZe", # it's possible they're large
               "https://open.spotify.com/playlist/30Is1VfjAKbU1iqv4gFgoV" # jimmy
              ]

In [ ]:
data = {'Username': [],
        'Name': [],
        'Datetime': [],
        'Name-Date': []
       }
  
# Convert the dictionary into DataFrame
df = pd.DataFrame(data)


In [ ]:
for uri in shared_urls:
    playlist_URI = uri.split("/")[-1].split("?")[0]
    track_uris = [x["track"]["uri"] for x in sp.playlist_tracks(playlist_URI)["items"]]
    
    for i in range(len(sp.playlist_tracks(playlist_URI)["items"])):
        
        username = sp.playlist_tracks(playlist_URI)["items"][i]["added_by"]["id"]
        name = username_dict[username]
        
        date_time = sp.playlist_tracks(playlist_URI)["items"][i]["added_at"]

        
        df = df.append({'Username' : username,
                        'Name': name,
                        'Datetime': date_time,
                        'Name-Date': f"{name}-{date_time.split('T')[0]}"},
                         ignore_index = True)

In [ ]:
summary_df = df['Name-Date'].value_counts()
summary_df = summary_df.to_frame()
summary_df = summary_df.rename(columns = {"Name-Date": "Count"})
summary_df.index.name = 'Name-Date'
summary_df.reset_index(inplace=True)

In [ ]:
date_format = '%Y-%m-%d'

summary_df['Name'] = [ name_date.split('-')[0] for name_date in summary_df['Name-Date']]
summary_df['Date'] = [ dt.datetime.strptime(f"{name_date.split('-')[1]}-{name_date.split('-')[2]}-{name_date.split('-')[3]}", date_format)
                         for name_date in summary_df['Name-Date']]

# Plotting

In [ ]:
# https://stackoverflow.com/questions/32485907/matplotlib-and-numpy-create-a-calendar-heatmap
import datetime as dt
import matplotlib.pyplot as plt
import numpy as np

def calendar_array(dates, data):
    i, j = zip(*[d.isocalendar()[1:] for d in dates])
    i = np.array(i) - min(i)
    j = np.array(j) - 1
    ni = max(i) + 1

    calendar = np.nan * np.zeros((7, ni))
    calendar[j,i ] = data
    return i, j, calendar


def calendar_heatmap(ax, dates, data, overall_max,
                     bg_color, fg_color,
                     show_color_bar = True):
    i, j, calendar = calendar_array(dates, data)
    im = ax.imshow(calendar, interpolation='none', cmap='summer',
                   vmin = 0, vmax = overall_max)

    label_days(ax, dates, i, j, calendar)
    label_months(ax, dates, i, j, calendar)
    
    ax.spines['bottom'].set_color(fg_color)
    ax.spines['top'].set_color(fg_color) 
    ax.spines['right'].set_color(fg_color)
    ax.spines['left'].set_color(fg_color)
    ax.xaxis.label.set_color(fg_color)
    ax.yaxis.label.set_color(fg_color)
    ax.tick_params(colors=fg_color, which='both')
    
    if show_color_bar:
        cb = ax.figure.colorbar(im, orientation="horizontal")
        cb.set_label('Number of songs added', color=fg_color)
        
        # set colorbar tick color
        cb.ax.xaxis.set_tick_params(color=fg_color, labelcolor=fg_color)

        # set colorbar edgecolor 
        cb.outline.set_edgecolor(bg_color)
        


def label_days(ax, dates, i, j, calendar):
#     ni, nj = calendar.shape
#     day_of_month = np.nan * np.zeros((ni, 7))
#     day_of_month[i, j] = [d.day for d in dates]

#     for (i, j), day in np.ndenumerate(day_of_month):
#         if np.isfinite(day):
#             ax.text(j, i, int(day), ha='center', va='center')

    ax.set(yticks=np.arange(7), 
           yticklabels=['M', 'T', 'W', 'R', 'F', 'S', 'S'])
    
    ax.yaxis.tick_left()

def label_months(ax, dates, i, j, calendar):
    month_labels = np.array(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul',
                             'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
    months = np.array([d.month for d in dates])
    uniq_months = sorted(set(months))
    xticks = [i[months == m].mean() for m in uniq_months]
    labels = [month_labels[m - 1] for m in uniq_months]
    ax.set(xticks=xticks)
    ax.set_xticklabels(labels, rotation=90)
    ax.xaxis.tick_top()

In [ ]:
def make_plot(ax, name, summary_df, bg_color, fg_color, show_color_bar = True): 
    
    
    overall_max = max(summary_df['Count'])
    min_date = min(summary_df['Date'])
    max_date = max(summary_df['Date'])
    
    subset_df = summary_df[summary_df['Name'] == name]
    total = sum(subset_df['Count'])
    
    # add entries for first and last overall day
    
    subset_df = subset_df.append({ 'Name': name,
                                   'Name-Date': f"{name}-{min_date}",
                                   'Date':  min_date,
                                   'Count': np.nan}, ignore_index=True)
    subset_df = subset_df.append({ 'Name': name,
                                   'Name-Date': f"{name}-{min_date}",
                                   'Date':  max_date,
                                   'Count': np.nan}, ignore_index=True)
    
#     fig, ax = plt.subplots(figsize=(6, 10))
    
#     fig, ax = plt.subplots(figsize=(12, 3))
    ax.set_title(f"{name} -- {total}", fontsize=25, color=fg_color,
                horizontalalignment = "center")


    calendar_heatmap(ax,
                     subset_df['Date'],
                     subset_df['Count'],
                     overall_max,
                     bg_color,
                     fg_color,
                     show_color_bar)
    
    ax.set_facecolor(bg_color)

In [ ]:
fig, axs = plt.subplots(nrows = 6,
                        figsize= (10, 17)
                       )
bg_color = "black"
fg_color = "lightgreen"

fig.suptitle("Playlist contributions by person", fontsize=40, color=fg_color, y = 1.00)
fig.set_facecolor(bg_color)

make_plot(axs[0], "Jimmy", summary_df, bg_color, fg_color, False)
make_plot(axs[1], "Ross", summary_df, bg_color, fg_color, False)
make_plot(axs[2], "Lukas", summary_df, bg_color, fg_color, False)
make_plot(axs[3], "Will", summary_df, bg_color, fg_color, False)
make_plot(axs[4], "Michael", summary_df, bg_color, fg_color, False)
make_plot(axs[5], "Gabe", summary_df, bg_color, fg_color, True)

plt.savefig('contributions_plot.png', facecolor=fig.get_facecolor(),)
